<a href="https://colab.research.google.com/github/Manmohan160/Deep-Learning-Project-samsung-/blob/Amrita/FinalKoEngage_Wiki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install datasets transformers sacrebleu bert_score scikit-learn optuna sacremoses langdetect

In [ ]:
# 1) Load and preprocess data from WikiMatrix.en-ko files
import pandas as pd
import re
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
import nltk
from langdetect import detect

nltk.download('punkt')
nltk.download('punkt_tab')
'''
# 1. Load the files
with open('WikiMatrix.en-ko.ko', 'r', encoding='utf-8') as f:
    ko_lines = [l.strip() for l in f if l.strip()]
with open('WikiMatrix.en-ko.en', 'r', encoding='utf-8') as f:
    en_lines = [l.strip() for l in f if l.strip()]

df = pd.DataFrame({'ko': ko_lines, 'en': en_lines})
print(f"✅ Loaded: {len(df)} sentence pairs")

# 2. Drop duplicates
df = df.drop_duplicates()
print(f"✅ After deduplication: {len(df)} pairs")

# 3. Language detection (filter out incorrect language pairs)
def is_lang(text, lang_code):
    try:
        return detect(text) == lang_code
    except:
        return False

df = df[df['en'].apply(lambda x: is_lang(x, 'en')) & df['ko'].apply(lambda x: is_lang(x, 'ko'))]
print(f"✅ After language filtering: {len(df)} pairs")

df.to_csv('wikimatrix_cleaned.csv', index=False, encoding='utf-8')
'''

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


'\n# 1. Load the files\nwith open(\'WikiMatrix.en-ko.ko\', \'r\', encoding=\'utf-8\') as f:\n    ko_lines = [l.strip() for l in f if l.strip()]\nwith open(\'WikiMatrix.en-ko.en\', \'r\', encoding=\'utf-8\') as f:\n    en_lines = [l.strip() for l in f if l.strip()]\n\ndf = pd.DataFrame({\'ko\': ko_lines, \'en\': en_lines})\nprint(f"✅ Loaded: {len(df)} sentence pairs")\n\n# 2. Drop duplicates\ndf = df.drop_duplicates()\nprint(f"✅ After deduplication: {len(df)} pairs")\n\n# 3. Language detection (filter out incorrect language pairs)\ndef is_lang(text, lang_code):\n    try:\n        return detect(text) == lang_code\n    except:\n        return False\n\ndf = df[df[\'en\'].apply(lambda x: is_lang(x, \'en\')) & df[\'ko\'].apply(lambda x: is_lang(x, \'ko\'))]\nprint(f"✅ After language filtering: {len(df)} pairs")\n\ndf.to_csv(\'wikimatrix_cleaned.csv\', index=False, encoding=\'utf-8\')\n'

In [ ]:
df = pd.read_csv('wikimatrix_cleaned.csv')

# 5. Length filtering: keep sentences with 2+ tokens and balanced lengths
df['len_ko'] = df['ko'].apply(lambda x: len(nltk.word_tokenize(x)))
df['len_en'] = df['en'].apply(lambda x: len(nltk.word_tokenize(x)))
ratio = df['len_en'] / df['len_ko']
df = df[(df['len_en'] > 2) & (df['len_ko'] > 2) & (ratio.between(0.5, 1.5))]
print(f"✅ After length filtering: {len(df)} pairs")

# 6. Drop helper columns
df = df.drop(columns=['len_ko', 'len_en'])

# 7. Down-sample to 30k
df = df.sample(n=30000, random_state=42).reset_index(drop=True)
print(f"After sampling: {len(df)} pairs")

# 8. Train/validation split
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)
raw_datasets = DatasetDict({
    'train': Dataset.from_pandas(train_df.reset_index(drop=True)),
    'validation': Dataset.from_pandas(val_df.reset_index(drop=True))
})
print("Final dataset:", raw_datasets)


✅ After length filtering: 177734 pairs
After sampling: 30000 pairs
Final dataset: DatasetDict({
    train: Dataset({
        features: ['ko', 'en'],
        num_rows: 27000
    })
    validation: Dataset({
        features: ['ko', 'en'],
        num_rows: 3000
    })
})


In [ ]:
# 3) Load tokenizer & model
import torch
from transformers import MarianTokenizer, MarianMTModel

model_name = 'Helsinki-NLP/opus-mt-ko-en'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name).to('cuda' if torch.cuda.is_available() else 'cpu')
device = model.device
print('Using device:', device)

Using device: cuda:0


In [ ]:
# 4) Sample translations before training
print('--- Pre-trained model samples ---')
for i in range(5):
    ko = raw_datasets['validation'][i]['ko']
    enc = tokenizer(ko, return_tensors='pt', truncation=True, max_length=128).to(device)
    out = model.generate(**enc, num_beams=4, max_length=128)
    print(f'KO: {ko}')
    print('EN:', tokenizer.decode(out[0], skip_special_tokens=True), '\n')

--- Pre-trained model samples ---
KO: 이 영화만을 위해 순수 작곡한 곡은 있지 않다.
EN: There are no pure compositions for this film alone. 

KO: 대멸종은 종종 지구상의 생명체의 진화를 가속하기도 한다.
EN: Great extinctions often accelerate the evolution of life on Earth. 

KO: 페르세포네가 하데스와 함께 있는 4달은 그리스의 건조한 여름에 해당하는데, 이 기간 동안 식물들은 가뭄으로 생존의 위협을 받는다.
EN: The four months that Percelle is with Hades are the dry summers of Greece, during which plants are threatened with survival because of drought. 

KO: 빅토리아 여왕의 샌드링엄 하우스에 두 차례 방문 중 한번은 그녀가 저녁 식사를 마치고 이곳에서 열린 파티를 열었는데, 그녀의 일기에 "채색된 천장과 두 개의 벽난로를 지닌 매우 길고 멋진 응접실"이라 적었다.
EN: On one of her two visits to Queen Victoria's Sandringham House, she had a party here after dinner, and in her diary she wrote: "a very long and wonderful parlour with a colored ceiling and two fireplaces." 

KO: 대시캠은 일반적으로 4G와 전면 360° 카메라를 채용한다.
EN: The dash usually employs a 4G camera and a front 360-degree camera. 



In [ ]:
# 5) Preprocess for training
from transformers import DataCollatorForSeq2Seq

max_len = 128
def preprocess_function(ex):
    in_tok = tokenizer(ex['ko'], truncation=True, padding='max_length', max_length=max_len)
    with tokenizer.as_target_tokenizer():
        lbl = tokenizer(ex['en'], truncation=True, padding='max_length', max_length=max_len)
    in_tok['labels'] = lbl['input_ids']
    return in_tok

tokenized = raw_datasets.map(preprocess_function, batched=True, remove_columns=['en','ko'])
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
print(tokenized)

Map:   0%|          | 0/27000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 27000
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3000
    })
})


In [ ]:
# 6) Metrics
!pip install evaluate
import evaluate
bleu = evaluate.load('bleu')
bertscore = evaluate.load('bertscore')
chrf = evaluate.load('chrf')

def compute_metrics(p):
    preds, labels = p
    dec_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    dec_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    return {
        'bleu': bleu.compute(predictions=dec_preds, references=[[l] for l in dec_labels])['bleu'],
        'bertscore': sum(bertscore.compute(predictions=dec_preds, references=dec_labels, lang='en')['f1'])/len(dec_labels),
        'chrf': chrf.compute(predictions=dec_preds, references=dec_labels)['score']
    }

In [ ]:

# 7) Hyperparameter search (Optuna)
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

def hp_space(trial):
    return {
        'learning_rate': trial.suggest_loguniform('learning_rate', 5e-6, 5e-5),
        'per_device_train_batch_size': trial.suggest_categorical('per_device_train_batch_size', [4,8]),
        'num_train_epochs': trial.suggest_int('num_train_epochs', 1, 3)
    }

args = Seq2SeqTrainingArguments(
    output_dir='temp',
    save_total_limit=1,
    predict_with_generate=True,
    fp16=torch.cuda.is_available(),
)

def model_init():
    return MarianMTModel.from_pretrained(model_name)
'''
trainer = Seq2SeqTrainer(
    model_init=model_init,
    args=args,
    train_dataset=tokenized['train'],
    eval_dataset=tokenized['validation'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

best_run = trainer.hyperparameter_search(
    direction='maximize',
    backend='optuna',
    n_trials=5,
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics['eval_bleu']
)
print('Best hyperparameters:', best_run.hyperparameters)
'''

"\ntrainer = Seq2SeqTrainer(\n    model_init=model_init,\n    args=args,\n    train_dataset=tokenized['train'],\n    eval_dataset=tokenized['validation'],\n    tokenizer=tokenizer,\n    data_collator=data_collator,\n    compute_metrics=compute_metrics\n)\n\nbest_run = trainer.hyperparameter_search(\n    direction='maximize',\n    backend='optuna',\n    n_trials=5,\n    hp_space=hp_space,\n    compute_objective=lambda metrics: metrics['eval_bleu']\n)\nprint('Best hyperparameters:', best_run.hyperparameters)\n"

In [ ]:
# Use the best hyperparameters directly
best_args = Seq2SeqTrainingArguments(
    output_dir='./best',
    learning_rate=5e-05,
    per_device_train_batch_size=8,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=torch.cuda.is_available(),  # Use FP16 if GPU available
    save_total_limit=1,
    logging_strategy="epoch"
)

# Re-initialize the model (optional: helps ensure clean training)
# Ensure model_init is defined as a function returning a fresh model instance
# Call model_init() to get the model instance
final_model = model_init()

# Train using Seq2SeqTrainer
trainer = Seq2SeqTrainer(
    model=final_model,  # Pass the final model instance here
    args=best_args,
    train_dataset=tokenized['train'],
    eval_dataset=tokenized['validation'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

print('### Evaluation before supervised fine-tuning ###')
print(trainer.evaluate())
trainer.train()
print('### Evaluation after supervised fine-tuning ###')
print(trainer.evaluate())

# 9) Sample translations after fine-tuning
print('--- Fine-tuned model samples ---')
# Use the fine-tuned model from the trainer
for i in range(5):
    ko = raw_datasets['validation'][i]['ko']
    enc = tokenizer(ko, return_tensors='pt', truncation=True, max_length=128).to(device)
    # Generate translations using the fine-tuned model instance
    out = final_model.generate(**enc, num_beams=4, max_length=128)
    print(f'KO: {ko}')
    print('EN:', tokenizer.decode(out[0], skip_special_tokens=True), '\n')

/tmp/ipython-input-20-1466630057.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


### Evaluation before supervised fine-tuning ###


{'eval_loss': 5.016435623168945, 'eval_model_preparation_time': 0.0052, 'eval_bleu': 0.154918445244799, 'eval_bertscore': 0.9100718971292178, 'eval_chrf': 44.08719157101856, 'eval_runtime': 540.8485, 'eval_samples_per_second': 5.547, 'eval_steps_per_second': 0.693}


Step,Training Loss
3375,0.504400


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3465: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[65000]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


### Evaluation after supervised fine-tuning ###
{'eval_loss': 0.449331670999527, 'eval_model_preparation_time': 0.0052, 'eval_bleu': 0.12729122731073006, 'eval_bertscore': 0.91150659430027, 'eval_chrf': 44.293792926034776, 'eval_runtime': 659.9381, 'eval_samples_per_second': 4.546, 'eval_steps_per_second': 0.568, 'epoch': 1.0}
--- Fine-tuned model samples ---
KO: 이 영화만을 위해 순수 작곡한 곡은 있지 않다.
EN: There is no pure song written for this film alone. 

KO: 대멸종은 종종 지구상의 생명체의 진화를 가속하기도 한다.
EN: Large extinctions often speed up the evolution of life on Earth. 

KO: 페르세포네가 하데스와 함께 있는 4달은 그리스의 건조한 여름에 해당하는데, 이 기간 동안 식물들은 가뭄으로 생존의 위협을 받는다.
EN: The four months that Perconene is with Hades are the dry summers of Greece, during which plants are threatened with survival due to drought. 

KO: 빅토리아 여왕의 샌드링엄 하우스에 두 차례 방문 중 한번은 그녀가 저녁 식사를 마치고 이곳에서 열린 파티를 열었는데, 그녀의 일기에 "채색된 천장과 두 개의 벽난로를 지닌 매우 길고 멋진 응접실"이라 적었다.
EN: On one of the two visits to Queen Victoria's Sandringham House, she hosted a party at the end 